# scikit-learnのトレーニング♨

## 単回帰、重回帰分析 編

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インポート](#インポート)
  - [プロキシ設定](#プロキシ設定)
  - [共通関数定義](#共通関数定義)
- [モデル・アルゴリズム](#モデル・アルゴリズム)
  - [単回帰分析](#単回帰分析)
  - [重回帰分析](#重回帰分析)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python#y4054d35

## [環境準備](ScikitLearnTraining0.ipynb)

### インポート

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets                                 # サンプル・データセット
from sklearn.datasets import make_regression                 # 回帰データセット作成
from sklearn.datasets import make_blobs                      # 分類データセット作成
from sklearn.preprocessing import StandardScaler             # 標準化
from sklearn.model_selection import train_test_split         # データ分割

from sklearn.linear_model import LinearRegression            # 線形回帰
from sklearn.preprocessing import PolynomialFeatures         # 多項式回帰の変数変換
from sklearn.linear_model import Ridge                       # 多項式回帰のRidge回帰
from sklearn.linear_model import Lasso                       # 多項式回帰のLASSO回帰
from sklearn.linear_model import ElasticNet                  # 多項式回帰のLASSO回帰
from sklearn.linear_model import Perceptron                  # 単純パーセプトロン線形分類器
from sklearn.linear_model import LogisticRegression          # ロジスティク回帰
from sklearn.svm import SVC                                  # サポートベクターマシン（SVM）分類器
from sklearn.tree import DecisionTreeClassifier              # 決定木（分類木）
from sklearn.ensemble import RandomForestClassifier          # ランダムフォレスト（分類木）
from sklearn.ensemble import GradientBoostingClassifier      # 勾配ブースティング木（分類木）
from sklearn.decomposition import PCA                        # 主成分分析
from sklearn.cluster import KMeans                           # k-means法 クラスタ分析
from sklearn.feature_extraction.text import CountVectorizer  # 自然言語処理ベクトル化
from sklearn.feature_extraction.text import TfidfTransformer # 自然言語ベクトルのTF-IDF計算
from sklearn.decomposition import LatentDirichletAllocation  # 自然言語ベクトルからLDAトピック抽出

from sklearn import metrics                                  # モデル評価
from sklearn.metrics import mean_squared_error as mse        # 精度評価（mse）
from sklearn.metrics import confusion_matrix                 # 混同行列
from sklearn.metrics import silhouette_samples               # シルエット係数
from sklearn.model_selection import cross_val_score          # 交差検証法
from sklearn.model_selection import KFold                    # k分割交差検証法
from sklearn.model_selection import StratifiedKFold          # 層化交差検証法
from sklearn.model_selection import GridSearchCV             # グリッドサーチ

import seaborn as sns                                        # matplotlibラッパ
from mlxtend.plotting import plot_decision_regions           # 決定領域表示関数
from matplotlib import cm                                    # カラーマップ処理

from numpy import linalg as LA                               # 線形代数ライブラリ

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### プロキシ設定

In [2]:
import os
proxies = {
    "http": os.getenv("HTTP_PROXY"),
    "https": os.getenv("HTTPS_PROXY")
}

### 共通関数定義

#### 残差プロットを行う関数

In [ ]:
def res_plot(y_train, y_train_pred, y_test, y_test_pred):
    # train, testごとに残差を算出
    res_train = y_train_pred - y_train
    res_test  = y_test_pred - y_test
    
    # train, testごとに(予測値, 残差)をプロット
    plt.figure(figsize=(8, 8))
    plt.scatter(y_train_pred, res_train, color='blue', marker='o', label='train', alpha=0.5)
    plt.scatter(y_test_pred, res_test, color='green', marker='s', label='test', alpha=0.5)
    
    plt.xlabel('Predicted Values')                  # x軸のラベルを表示
    plt.ylabel('Residuals')                         # y軸のラベルを表示
    plt.legend(loc='upper left')                    # 凡例を左上に表示
    plt.hlines(y=0, xmin=-10, xmax=50, color='red') # 残差0の線を表示
    plt.xlim([-10, 50])                             # 表示する横軸の範囲を指定
    plt.show()

## モデル・アルゴリズム

### 単回帰分析
- 線形単回帰分析（分布を直線の１次関数で近似
- 最小二乗法で直線の１次関数を求める。

※ [イメージ](https://www.google.com/search?q=単回帰分析&tbm=isch)、[説明](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E7%B5%B1%E8%A8%88%E8%A7%A3%E6%9E%90#a421ea17)

#### データ

##### 生成
- その日の最高気温と売れたアイスの個数のデータを利用。
- 必要に応じて[環境変数にプロキシ設定](#プロキシ設定)をしておくと良い。

In [ ]:
url = 'https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?plugin=attach&pcmd=open&file=n_ice%2Bvs%2Btemp.xlsx&refer=FrontPage'
res = requests.get(url) # , verify=False, proxies=proxies) # プロキシ
df = pd.read_excel(io.BytesIO(res.content), engine='openpyxl')
df

##### 加工

In [ ]:
df=df.drop('Unnamed: 0',axis=1)
df

##### 理解

In [ ]:
# 基本統計量
df.describe()

In [ ]:
# 相関係数（相関行列）
df.corr()

In [ ]:
# ヒートマップに表示
plt.figure(figsize=(4, 4))
sns.heatmap(df.corr(), annot=True, square=True, fmt='.2f')
plt.show()

In [ ]:
# 散布図行列を表示
sns.pairplot(df)
plt.show()

##### 準備

In [ ]:
# 説明変数・目的変数の選択・分割
x = np.array(df.loc[:, ['MAX_TEMP']])
y = np.array(df.loc[:, ['N_ICE']])
# データをスライシングして確認。
print(x[:5])
print(y[:5])

In [ ]:
# 学習・テストデータの分割（ホールド・アウト法
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

#### モデリング

##### 実行

###### 学習
※ fitは多変量に対応しているので二次元配列であること。

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)

lr = LinearRegression()
lr.fit(x, y)

###### 推論
※ predictは多変量に対応しているので二次元配列であること。

In [ ]:
index=6
new_val = np.array([x_test[index]])
pred_val = lr.predict(new_val)
ratio_val = y_test[index] / pred_val[0]
print('answer = ', y_test[index])
print('pred = ', pred_val[0])
print('ratio : %.2f' % ratio_val)

##### 評価

###### 回帰直線
y = lr.coef_[0] * x + lr.intercept_

In [ ]:
print('coefficient = ', lr.coef_[0]) # 係数
print('intercept = ', lr.intercept_) # 切片

###### 決定係数（R2乗値）
絶対的な基準でないが0.5以上であれば適合度が高い

In [ ]:
print('R^2')
print('train: %.3f' % lr.score(x_train, y_train))
print('test : %.3f' % lr.score(x_test, y_test))

###### 散布図に回帰直線を追加

In [ ]:
plt.scatter(x, y, color = 'blue')
plt.plot(x, lr.predict(x), color = 'red')
plt.grid()
plt.show()

### 重回帰分析
- 線形重回帰分析（分布を（超）平面などの１次関数で近似
- ３変数以上の最小二乗法は勾配降下法で解く

※ [イメージ](https://www.google.com/search?q=重回帰分析&tbm=isch)、[説明](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E7%B5%B1%E8%A8%88%E8%A7%A3%E6%9E%90#na76c5ec)

#### データ

##### 生成

###### ダウンロードしてDFに読込
必要に応じて[環境変数にプロキシ設定](#プロキシ設定)をしておくと良い。

In [ ]:
# Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.
'''
## bostonデータを格納したクラス
from sklearn.datasets import load_boston
## インスタンスを生成
boston = load_boston()
## データの詳細を出力
print(boston.DESCR)
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df
'''

In [ ]:
url = 'https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?plugin=attach&pcmd=open&file=Boston.csv&refer=FrontPage'
res = requests.get(url) # , verify=False, proxies=proxies) # プロキシ
df = pd.read_csv(io.BytesIO(res.content), encoding='utf-8', sep=",")
df

###### [説明を参照](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20DataSet#j8cfe8df)

In [ ]:
df.columns

##### 加工

In [ ]:
df=df.drop('Unnamed: 0',axis=1)
df

In [ ]:
# 特徴量の数が多いので､locメソッドで一部の特徴量のみを選択
df = df.loc[:, ['lstat', 'indus', 'dis', 'rm', 'medv']]

##### 理解

In [ ]:
# 基本統計量
df.describe()

In [ ]:
# 相関係数（相関行列）
df.corr()

In [ ]:
# ヒートマップに表示
plt.figure(figsize=(5, 5))
sns.heatmap(df.corr(), annot=True, square=True, fmt='.2f')
plt.show()

In [ ]:
# 散布図行列を表示
sns.pairplot(df)
plt.show()

##### 準備

In [ ]:
# 説明変数・目的変数の選択・分割
# x = np.array(df.drop(['medv'], axis=1))
x = np.array(df.drop(['indus', 'dis', 'medv'], axis=1))
y = np.array(df.loc[:, ['medv']])
# データをスライシングして確認。
print(x[:5])
print(y[:5])

In [ ]:
# 偏回帰係数を見て、追加の特徴量エンジニアリングを行う場合
# 説明変数に標準化を施す必要がある。
'''
ss = StandardScaler()

# 引数は numpy.ndarray
x = ss.fit_transform(x)

# 確認
## 平均が ≒ 0
print("x.mean(): ", x.mean())
## 標準偏差が ≒ 1
print("x.std(): ", x.std())
'''

In [ ]:
# 学習・テストデータの分割（ホールド・アウト法
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

#### モデリング

##### 実行

###### 学習

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)

###### 推論
※ predictは多変量に対応しているので二次元配列であること。

In [ ]:
index=6
new_val = np.array([x_test[index]])
pred_val = lr.predict(new_val)
ratio_val = y_test[index] / pred_val[0]
print('answer = ', y_test[index])
print('pred = ', pred_val[0])
print('ratio : %.2f' % ratio_val)

##### 評価

###### 重回帰超平面
- y = lr.coef_[0] * x1 + lr.coef_[1] * x2 * ... + lr.intercept_
- 偏回帰係数は、
  - データの標準化を施してあれば、追加の特徴量エンジニアリングで参考にできる。
  - ただし、抑制変数があるので、最終的には[R2乗値](#自由度調整済み決定係数（R2乗値）)を参考にする。

In [ ]:
print('coefficient = ', lr.coef_) # 偏回帰係数
print('intercept = ', lr.intercept_) # 定数項

###### 決定係数（R2乗値）
絶対的な基準でないが0.5以上であれば適合度が高い

In [ ]:
print('R^2')
print('train: %.3f' % lr.score(x_train, y_train))
print('test : %.3f' % lr.score(x_test, y_test))

###### 自由度調整済み決定係数（R2乗値）

In [ ]:
def adjusted(score, n_sample, n_explanatory_variables):
    adjusted_score = 1 - (1 - score) * ((n_sample - 1) / (n_sample - n_explanatory_variables - 1))
    return adjusted_score

print('train: %.3f' % adjusted(lr.score(x_train, y_train), len(y_train), x_train.shape[1]))
print('test : %.3f' % adjusted(lr.score(x_test, y_test), len(y_test), x_test.shape[1]))

###### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)
- 平均二乗誤差（MSE：Mean Squared Error）
- 誤差の標準偏差（RMSE : Root Mean Squared Error）

In [ ]:
train_mse = mse(y_train, lr.predict(x_train))
test_mse = mse(y_test, lr.predict(x_test))

print('train_mse: %.3f' % train_mse)
print('test_mse: %.3f' % test_mse)
print('train_rmse: %.3f' % (train_mse ** (1/2)))
print('test_rmse : %.3f' % (test_mse ** (1/2)))

###### 重回帰超平面の可視化
ただし、説明変数が２変数まで。

In [ ]:
# 可視化

# import
from mpl_toolkits.mplot3d import Axes3D

# 平面のグリッド座標
w0 = lr.intercept_
w1 = lr.coef_[0, 0]
w2 = lr.coef_[0, 1]
x, y = np.meshgrid(np.arange(0, 40, 2.5), np.arange(1, 10, 0.5))
z = w1*x + w2*y + w0 

# 図表の定義
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# 3Dで Scat & plot
ax.scatter3D(df['lstat'], df['rm'], df['medv'])
ax.plot_wireframe(x, y, z)

# ラベル
ax.set_xlabel('lstat')
ax.set_ylabel('rm')
ax.set_zlabel('medv')

###### 残差プロットを行う。
- 説明変数が３変数以上でも適用できる。
- 参考：https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#i3d85f2b

In [ ]:
# 残差プロットを出力
res_plot(y_train, lr.predict(x_train), y_test, lr.predict(x_test))